In [0]:
from math import radians, cos, sin, asin, sqrt
from pyspark.sql import Window
import pyspark.sql.functions as F

## data

In [0]:
path = "/mnt/processed-dev/hist/Daily/CutRide/2021/*/*/"
data = spark.read.parquet(path).select("dateentry","rideid","latitude","longitude","deviceid","calculated_speed")

In [0]:
#fonction pour calculer la distance
@udf("double")
def get_distance(longit_a, latit_a, longit_b, latit_b):
  if longit_a is None or latit_a is None or longit_b is None or latit_b is None:
    return 0
  # Transform to radians
  longit_a, latit_a, longit_b, latit_b = map(radians, [longit_a, latit_a, longit_b, latit_b])
  dist_longit = longit_b - longit_a
  dist_latit = latit_b - latit_a
  # Calculate area
  area = sin(dist_latit / 2) ** 2 + cos(latit_a) * cos(latit_b) * sin(dist_longit / 2) ** 2
  # Calculate the central angle
  central_angle = 2 * asin(sqrt(area))
  radius = 6371000
  # Calculate Distance in meter
  distance = central_angle * radius
  return abs(distance)

In [0]:
#preparer les données pour calculer la distance de chaque trajet avec la fonction(lag)
def data_distance(data):
  w = Window().partitionBy("rideid").orderBy(F.col("deviceid").asc(), F.col("dateentry").asc())
  data_2 = data
  data_2 = data_2.select("*", F.lag("dateentry").over(w).alias("previousDateentry")).na.drop()
  data_2 = data_2.select("*", F.lag("deviceid").over(w).alias("previousDeviceid")).na.drop()
  data_2 = data_2.select("*", F.lag("latitude").over(w).alias("previousLatitude")).na.drop()
  data_2 = data_2.select("*", F.lag("longitude").over(w).alias("previousLongitude")).na.drop()
  data_2 = data_2.select("*", F.lag("calculated_speed").over(w).alias("previousSpeed")).na.drop()
  data_distance= data_2.withColumn("distance",get_distance(data_2.longitude, data_2.latitude, data_2.previousLongitude, data_2.previousLatitude))
  return data_distance

In [0]:
#ajouter la colonne distance en Km
data_brut = data_distance(data)
data_brut = data_brut.withColumn("distance_km",F.col("distance")/1000)

In [0]:
#Convert dateentry to date
data_brut=(data_brut.withColumn("date", F.to_date(F.from_unixtime(F.col('dateentry')/1000))))

## data ride

In [0]:
#calcule point de depart et arrivé pour trajet pour un deviceid
def point_start_end(data):
 
  windows = Window.partitionBy("rideid","deviceid").orderBy(F.col("dateentry").asc())

  df = (
           data.withColumn('long_end', F.last('longitude').over(windows))
            .withColumn('long_start', F.first('longitude').over(windows))
            .withColumn('lat_start', F.first('latitude').over(windows))
            .withColumn('lat_end', F.last('latitude').over(windows))

         )
  DF=df.groupby("rideid","deviceid","long_start","lat_start").agg(F.last("long_end").alias("long_end"),
                                          (F.last("lat_end").alias("lat_end")),(F.sum("distance_km").alias("distance_trajet")),(F.avg("calculated_speed").alias("vitesse_moy")),F.first("date").alias('date'))
  return DF


In [0]:
data_ride = point_start_end(data_brut)

In [0]:
dataframe_dico = {}
dataframe_dico["rideid_table_raw"]=data_ride

## Trajet Table

In [0]:
#rendre tt les tajets dans le mm sens 
def trajet_mm_sens(DF):
  tmp=DF.select('*')
  tmp2=(tmp.withColumn('lat_start2',F.when(tmp.lat_start > tmp.lat_end,tmp.lat_end).otherwise(tmp.lat_start))
          .withColumn('long_start2',F.when(tmp.lat_start > tmp.lat_end,tmp.long_end).otherwise(tmp.long_start))
          .withColumn('lat_end2',F.when(tmp.lat_end < tmp.lat_start,tmp.lat_start).otherwise(tmp.lat_end))
          .withColumn('long_end2',F.when(tmp.lat_end < tmp.lat_start,tmp.long_start).otherwise(tmp.long_end))
     )
  tmp3=tmp2.drop(tmp2.lat_start).drop(tmp2.lat_end).drop(tmp2.long_end).drop(tmp2.long_start)
  return tmp3

In [0]:
data_trajet = trajet_mm_sens(data_ride)

In [0]:
# rassembler tt les trajet avec 1km pres,111 metres pres suivant le parametre de "a" pour definir le round 
def  func_round(data,a):
  tmp=data.select('rideid','deviceid','distance_trajet',(F.round('long_start2',a)).alias('long_start'),(F.round('lat_start2',a)).alias('lat_start'),
                (F.round('long_end2',a)).alias('long_end'),(F.round('lat_end2',a)).alias('lat_end'))

  return  tmp

In [0]:
data_trajet=func_round(data_trajet,2)

In [0]:
#pour chaque deviceid on affiche  la liste de rideid , point de depart et arrivé et la frequence
data_trajet =data_trajet.groupby("deviceid",'long_start','lat_start','long_end','lat_end').agg(F.collect_list('rideid').alias("list_rideid"))
data_trajet = data_trajet.withColumn("type_ride_Id", F.monotonically_increasing_id())
data_trajet = data_trajet.select("*",F.explode("list_rideid").alias("rideid"))
                                                                                

In [0]:
data_trajet.display()

deviceid,long_start,lat_start,long_end,lat_end,list_rideid,type_ride_Id,rideid
0069D476-5E46-4924-ADE8-7EC71DF8E365#ANONYMOUSTOKEN3695,6.18,47.63,6.18,47.63,List(3a68665c-8849-4abb-b41c-fac612ca7ddb_0069D476-5E46-4924-ADE8-7EC71DF8E365#ANONYMOUSTOKEN3695),0,3a68665c-8849-4abb-b41c-fac612ca7ddb_0069D476-5E46-4924-ADE8-7EC71DF8E365#ANONYMOUSTOKEN3695
00c684595891aba0#77775073,2.32,48.83,2.3,48.84,List(6160b769-7fb5-4183-a882-812266ec6f04_00c684595891aba0#77775073),1,6160b769-7fb5-4183-a882-812266ec6f04_00c684595891aba0#77775073
00c684595891aba0#77775073,4.25,39.89,4.14,39.94,List(0be3b56c-254f-4db0-badd-adfa6d72208f_00c684595891aba0#77775073),2,0be3b56c-254f-4db0-badd-adfa6d72208f_00c684595891aba0#77775073
0123456789ABCDEF#41552778,7.69,48.58,7.74,48.6,List(cc67eaef-e36e-46d0-b158-aa8f5d9f71d6_0123456789ABCDEF#41552778),3,cc67eaef-e36e-46d0-b158-aa8f5d9f71d6_0123456789ABCDEF#41552778
01443DA1-6B10-4B1C-9D23-716EB28D133F#74728137,5.48,43.3,5.48,43.33,List(021ebe6e-912e-41b2-9f4d-82de4680182b_01443DA1-6B10-4B1C-9D23-716EB28D133F#74728137),4,021ebe6e-912e-41b2-9f4d-82de4680182b_01443DA1-6B10-4B1C-9D23-716EB28D133F#74728137
01443DA1-6B10-4B1C-9D23-716EB28D133F#ANONYMOUSTOKEN3138,5.47,43.33,5.51,43.39,List(108b7009-0f3e-49b1-b368-8fbdba53e66f_01443DA1-6B10-4B1C-9D23-716EB28D133F#ANONYMOUSTOKEN3138),5,108b7009-0f3e-49b1-b368-8fbdba53e66f_01443DA1-6B10-4B1C-9D23-716EB28D133F#ANONYMOUSTOKEN3138
01FCB53B-AC37-4DAA-AA1F-B1D60BE18F29#90126046,4.08,45.6,4.1,45.61,List(1fee700a-c2b7-4576-8b3d-2fa2780a4206_01FCB53B-AC37-4DAA-AA1F-B1D60BE18F29#90126046),6,1fee700a-c2b7-4576-8b3d-2fa2780a4206_01FCB53B-AC37-4DAA-AA1F-B1D60BE18F29#90126046
01FCB53B-AC37-4DAA-AA1F-B1D60BE18F29#90126046,4.14,45.74,4.22,45.74,List(7d8e20e8-c0ce-4616-a349-77ad1c8017c4_01FCB53B-AC37-4DAA-AA1F-B1D60BE18F29#90126046),7,7d8e20e8-c0ce-4616-a349-77ad1c8017c4_01FCB53B-AC37-4DAA-AA1F-B1D60BE18F29#90126046
01FCB53B-AC37-4DAA-AA1F-B1D60BE18F29#90126046,4.22,45.74,4.0,45.75,"List(2058a116-7c51-4753-bcf3-eba624d6e4d8_01FCB53B-AC37-4DAA-AA1F-B1D60BE18F29#90126046, 5891320a-1357-49f3-816b-4ab9774b1d34_01FCB53B-AC37-4DAA-AA1F-B1D60BE18F29#90126046)",8,2058a116-7c51-4753-bcf3-eba624d6e4d8_01FCB53B-AC37-4DAA-AA1F-B1D60BE18F29#90126046
01FCB53B-AC37-4DAA-AA1F-B1D60BE18F29#90126046,4.22,45.74,4.0,45.75,"List(2058a116-7c51-4753-bcf3-eba624d6e4d8_01FCB53B-AC37-4DAA-AA1F-B1D60BE18F29#90126046, 5891320a-1357-49f3-816b-4ab9774b1d34_01FCB53B-AC37-4DAA-AA1F-B1D60BE18F29#90126046)",8,5891320a-1357-49f3-816b-4ab9774b1d34_01FCB53B-AC37-4DAA-AA1F-B1D60BE18F29#90126046


In [0]:
#Convert array column to a String with "concat_ws"
#concat_ws(sep, *cols)
data_trajet=data_trajet.withColumn("list_rideid", F.concat_ws(" ,", "list_rideid"))

In [0]:
data_trajet.write.option('header',True).mode('overwrite').csv('/mnt/datalake/tmp/amani/id_type_trajet_data/2021_id_type_ride')

##jointure

In [0]:
final_data_ride = (data_ride.join(data_trajet.select("rideid","type_ride_id"), on= data_ride.rideid == data_trajet.rideid,how = "left")
                            .drop(data_trajet.rideid))


In [0]:
final_data_ride.write.option('header',True).mode('overwrite').csv('/mnt/datalake/tmp/amani/ride_data/2021_ride')

In [0]:
final_data_ride.display()

rideid,deviceid,long_start,lat_start,long_end,lat_end,distance_trajet,vitesse_moy,date,type_ride_id
157c0bf7-d85e-4f36-8d29-f3d6f4269c06_CC8659D2-3FF4-4276-8626-4D2B3BE15003#13365587,CC8659D2-3FF4-4276-8626-4D2B3BE15003#13365587,0.5322829312225938,44.85209276491515,0.5706434903653865,44.86494920553644,6.410783227638797,2.724104565404895,2021-04-03,34359742443
19390789-299b-4c5f-8f0c-169ad19ea601_83b62548047ef262#akinot10,83b62548047ef262#akinot10,1.1769849,49.2413041,1.223806,49.2752772,7.578016074655895,10.293349861480811,2021-12-18,17179870922
2f310a19-33e5-445b-8096-66dc040b3390_A3F898D5-A2BE-4F92-BB7A-D0E0C205B063#hg99lw10,A3F898D5-A2BE-4F92-BB7A-D0E0C205B063#hg99lw10,2.126696378086457,49.048030305611114,2.1738147562225976,49.07065476789803,7.847538622419937,4.134615405046095,2021-10-07,34359741840
5c76ce23-0355-48e2-a6c8-2f7b97318b3b_B4C62706-DD0D-4B18-A12D-19327906FBC4#ANONYMOUSTOKEN3232,B4C62706-DD0D-4B18-A12D-19327906FBC4#ANONYMOUSTOKEN3232,5.537732833793865,43.21385887836283,5.500766903286606,43.20082238941542,6.368957845761858,2.1939553684089406,2021-07-31,25769805880
9bd5281b-af88-493f-acbe-103c3b732a80_abd29d59922fbb67#ANONYMOUSTOKEN4234,abd29d59922fbb67#ANONYMOUSTOKEN4234,-0.1163709,46.3613522,-0.3675606,46.3602991,23.50637421009157,17.769762898821625,2021-11-12,8589937656
00f0dd70-9db6-474f-b6e9-8af89639ad92_unknown_1620334414#33553055,unknown_1620334414#33553055,-1.6550345,48.0914259,-1.6530251,48.0854146,1.2399415276393135,5.579841070862914,2021-11-10,34359744502
0e46607f-0ae9-46a2-a552-aa515fd0fcf3_60AC643E-2E23-4DF9-84E7-2417AE3692E6#07544203,60AC643E-2E23-4DF9-84E7-2417AE3692E6#07544203,6.328649899999999,46.3697399,5.825353255746438,46.11219094063042,73.21337738375726,13.892000949413852,2021-02-13,34359740684
283bd461-ead2-46f0-ae6c-750427cd33c0_306AEB0E-E769-4825-8997-A141F817130D#83653554,306AEB0E-E769-4825-8997-A141F817130D#83653554,1.9922443105652206,46.20633710611662,1.8947238178560137,46.17755720215805,16.61505426902404,8.017610676520462,2021-05-20,25769804238
45716ee1-60bc-4bed-92ba-1039c7d9083b_362FCE1E-5679-4DB9-B351-39BD582D1EBE#sxjeop10,362FCE1E-5679-4DB9-B351-39BD582D1EBE#sxjeop10,4.791883165021233,45.691118929577705,4.806693568830198,45.70192144256616,3.3092517219219433,3.721486490205161,2021-03-15,34359739404
6eae5274-fc7c-491d-a3bd-4ea34f48c154_A44B56C4-2904-49A8-AC4D-A8170B81BFD6#ANONYMOUSTOKEN3446,A44B56C4-2904-49A8-AC4D-A8170B81BFD6#ANONYMOUSTOKEN3446,5.404863152469949,43.305394877403636,5.409841212666891,43.30515024462001,0.7862732043591674,2.3200873448859505,2021-04-19,25769805760
